## This code is used to analysis each model MACs info as well as the simulation results

In [1]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from d2l import torch as d2l
import numpy as np
import pandas as pd
from ptflops import get_model_complexity_info
from pathlib import Path
import subprocess
import os
import time

### Read all Models

#### Alexnet

In [8]:
def alexnet(img_channel, num_labels):
    net = nn.Sequential(
        # 这里使用一个11*11的更大窗口来捕捉对象。
        # 同时，步幅为4，以减少输出的高度和宽度。
        # 另外，输出通道的数目远大于LeNet
        nn.Conv2d(img_channel, 64, kernel_size=11, stride=4, padding=2), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        # 减小卷积窗口，使用填充为2来使得输入与输出的高和宽一致，且增大输出通道数
        nn.Conv2d(64, 192, kernel_size=5, padding=2), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        # 使用三个连续的卷积层和较小的卷积窗口。
        # 除了最后的卷积层，输出通道的数量进一步增加。
        # 在前两个卷积层之后，汇聚层不用于减少输入的高度和宽度
        nn.Conv2d(192, 384, kernel_size=3, padding=1), nn.ReLU(),
        nn.Conv2d(384, 256, kernel_size=3, padding=1), nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, padding=1), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.AdaptiveAvgPool2d((6, 6)),   # 使用全局平均池化对每个通道中所有元素求平均并直接将结果传递到全连接层
        nn.Flatten(),
        # 这里，全连接层的输出数量是LeNet中的好几倍。使用dropout层来减轻过拟合
        nn.Linear(256 * 6 * 6, 4096), nn.ReLU(),
        nn.Dropout(p=0.5),
        nn.Linear(4096, 4096), nn.ReLU(),
        nn.Dropout(p=0.5),
        # 最后是输出层。由于这里使用Fashion-MNIST，所以用类别数为10，而非论文中的1000
        nn.Linear(4096, num_labels))
    return net

# print the model structure
AlexNet = alexnet(1, 10)    
# print(net)
# print each layer
alexlayer = []
DNNlayer = []
num = 0
for layer in AlexNet:
    num += 1
    name = layer.__class__.__name__
    layer_name = name[:4] + str(num)
    DNNlayer.append(name)
    alexlayer.append(layer_name)
# find the unique layer name, and fix the order
DNNlayer_org = sorted(list(set(DNNlayer)))
print('The layer name is:', alexlayer)
print('The layer name after orged is:', DNNlayer_org)
# the number of layers, which contains ['AdaptiveAvgPool2d', 'Conv2d', 'Dropout', 'Flatten', 'Linear', 'MaxPool2d', 'ReLU']
alexlayer_num = len(alexlayer) 
print('The number of layers is:', alexlayer_num)

The layer name is: ['Conv1', 'ReLU2', 'MaxP3', 'Conv4', 'ReLU5', 'MaxP6', 'Conv7', 'ReLU8', 'Conv9', 'ReLU10', 'Conv11', 'ReLU12', 'MaxP13', 'Adap14', 'Flat15', 'Line16', 'ReLU17', 'Drop18', 'Line19', 'ReLU20', 'Drop21', 'Line22']
The layer name after orged is: ['AdaptiveAvgPool2d', 'Conv2d', 'Dropout', 'Flatten', 'Linear', 'MaxPool2d', 'ReLU']
The number of layers is: 22
